# Load Packages

In [1]:
import zstandard as zstd
import io
import re
import time
import pandas as pd
import chess

# Cleaning the data

In [67]:
#Example game 

# [Event "Rapid swiss https://lichess.org/swiss/7siKNYGN"]
# [Site "https://lichess.org/ECy8BFok"]
# [Date "2025.03.01"]
# [Round "-"]
# [White "STRI0006"]
# [Black "PORT0024"]
# [Result "1-0"]
# [UTCDate "2025.03.01"]
# [UTCTime "00:00:00"]
# [WhiteElo "939"]
# [BlackElo "404"]
# [WhiteRatingDiff "+3"]
# [BlackRatingDiff "-4"]
# [ECO "C41"]
# [Opening "Philidor Defense"]
# [TimeControl "600+0"]
# [Termination "Normal"]

# 1. e4 { [%eval 0.18] [%clk 0:10:00] } 1... e5 { [%eval 0.21] [%clk 0:10:00] } 2. Nf3 { [%eval 0.13] [%clk 0:09:59] } 2... d6 { [%eval 0.48] [%clk 0:10:00] } 3. Nc3 { [%eval 0.28] [%clk 0:09:57] } 3... Nc6 { [%eval 0.56] [%clk 0:09:52] } 4. Bc4 { [%eval 0.29] [%clk 0:09:55] } 4... Be6 { [%eval 0.42] [%clk 0:09:43] } 5. Bxe6 { [%eval 0.41] [%clk 0:09:53] } 5... fxe6 { [%eval 0.33] [%clk 0:09:42] } 6. d3 { [%eval 0.19] [%clk 0:09:49] } 6... d5? { [%eval 1.56] [%clk 0:09:29] } 7. exd5?! { [%eval 0.92] [%clk 0:09:48] } 7... Nb4? { [%eval 3.42] [%clk 0:09:02] } 8. dxe6 { [%eval 3.32] [%clk 0:09:42] } 8... Nxd3+? { [%eval 6.02] [%clk 0:08:59] } 9. cxd3 { [%eval 6.29] [%clk 0:09:40] } 9... Bb4 { [%eval 8.18] [%clk 0:08:46] } 10. Bg5 { [%eval 6.43] [%clk 0:09:36] } 10... Qd7?? { [%eval 14.09] [%clk 0:08:42] } 11. exd7+ { [%eval 13.67] [%clk 0:09:35] } 11... Kxd7 { [%eval 12.74] [%clk 0:08:40] } 12. Nxe5+ { [%eval 12.99] [%clk 0:09:34] } 12... Kc8?! { [%eval #4] [%clk 0:08:31] } 13. Qg4+ { [%eval #3] [%clk 0:09:30] } 13... Kb8 { [%eval #3] [%clk 0:08:24] } 14. O-O-O?! { [%eval 12.06] [%clk 0:09:22] } 14... b6 { [%eval 12.62] [%clk 0:08:21] } 15. Qxb4 { [%eval 12.49] [%clk 0:09:10] } 15... Nf6 { [%eval 12.61] [%clk 0:08:10] } 16. Nf7 { [%eval 11.93] [%clk 0:09:05] } 16... Rc8 { [%eval 12.35] [%clk 0:08:05] } 17. Bf4 { [%eval 12.26] [%clk 0:08:57] } 17... a5?! { [%eval #1] [%clk 0:07:58] } 18. Qe4?! { [%eval 10.01] [%clk 0:08:54] } 18... Nxe4 { [%eval 10.11] [%clk 0:07:49] } 19. dxe4 { [%eval 10.11] [%clk 0:08:53] } 19... g5 { [%eval 10.38] [%clk 0:07:42] } 20. Bxg5 { [%eval 10.05] [%clk 0:08:49] } 20... h5 { [%eval 10.31] [%clk 0:07:41] } 21. Bd2 { [%eval 9.51] [%clk 0:08:42] } 21... Ra6 { [%eval 10.13] [%clk 0:07:36] } 22. Bf4 { [%eval 10.14] [%clk 0:08:40] } 22... Kb7 { [%eval 10.32] [%clk 0:07:30] } 23. Rd8 { [%eval 9.93] [%clk 0:08:31] } 23... Rxd8 { [%eval 10.16] [%clk 0:07:27] } 24. Nxd8+ { [%eval 10.19] [%clk 0:08:30] } 24... Kc8 { [%eval 10.09] [%clk 0:07:23] } 25. Nc6 { [%eval 9.96] [%clk 0:08:26] } 25... Kb7 { [%eval 9.94] [%clk 0:07:20] } 26. Ne7 { [%eval 9.76] [%clk 0:08:19] } 26... h4 { [%eval 10.77] [%clk 0:07:16] } 27. e5 { [%eval 9.56] [%clk 0:08:06] } 27... c6 { [%eval 11.05] [%clk 0:07:13] } 28. Ng6 { [%eval 10.05] [%clk 0:07:52] } 28... a4 { [%eval 10.75] [%clk 0:07:06] } 29. e6 { [%eval 10.64] [%clk 0:07:49] } 29... a3 { [%eval 10.54] [%clk 0:07:02] } 30. bxa3 { [%eval 10.52] [%clk 0:07:48] } 30... Rxa3 { [%eval 10.15] [%clk 0:07:00] } 31. Be5 { [%eval 10.21] [%clk 0:07:43] } 31... Rxa2?! { [%eval #8] [%clk 0:06:57] } 32. Nxa2 { [%eval #7] [%clk 0:07:41] } 32... c5 { [%eval #7] [%clk 0:06:54] } 33. e7 { [%eval #6] [%clk 0:07:40] } 33... b5 { [%eval #4] [%clk 0:06:53] } 34. e8=Q { [%eval #3] [%clk 0:07:38] } 34... c4 { [%eval #3] [%clk 0:06:48] } 35. Qxb5+ { [%eval #2] [%clk 0:07:36] } 35... Ka7 { [%eval #2] [%clk 0:06:46] } 36. Qb8+ { [%eval #2] [%clk 0:07:31] } 36... Ka6 { [%eval #2] [%clk 0:06:43] } 37. Nb4+ { [%eval #2] [%clk 0:07:28] } 37... Ka5 { [%eval #2] [%clk 0:06:41] } 38. Bc7+ { [%eval #2] [%clk 0:07:25] } 38... Ka4 { [%eval #2] [%clk 0:06:39] } 39. Nc2 { [%eval #1] [%clk 0:07:22] } 39... c3 { [%eval #1] [%clk 0:06:35] } 40. Bd6 { [%eval #2] [%clk 0:07:20] } 40... h3 { [%eval #1] [%clk 0:06:32] } 41. Qb4# { [%clk 0:07:19] } 1-0

In [68]:
# Configuration constants
input_path = "sample_games.pgn"
output_path = "games_clean_sample.csv"
min_elo = 1000 #Note: in our cleaning we set this parmeter to 2600

csv_headers = "ID;Result;WhiteElo;BlackElo;ECO;Opening;TimeControl;Termination;Moves\n"
keep = ("[Site ", "[Result ", "[WhiteElo ", "[BlackElo ", 
        "[ECO ", "[Opening ", "[TimeControl ", "[Termination ", "1.")

In [69]:
def extract_value(line):
    """Extract the quoted value from a PGN header line."""
    # e.g. '[White "STRI0006"]' -> 'STRI0006'
    return line.split("\"")[1]

In [70]:
def clean_moves(line):
    """Strip the moves of annotation"""
    #e.g. 1. e4 { [%eval 0.18] [%clk 0:10:00] } 1... e5 { [%eval 0.21] [%clk 0:10:00] } 2. Nf3 { [%eval 0.48] [%clk 0:10:00] } 1-0
    #–> 1. e4 e5 2. Nf3 1-0
    line = re.sub(r"\s\{[^{}]*\}|[!?]+", "", line)
    return re.sub(r'\s+\d+\.\.\.\s+', ' ', line).strip()

In [ ]:
def extract_games(input_path, output_path, min_elo, keep, csv_headers):
    start = time.time()
    with open(input_path, "rb") as in_file:
        text_stream = io.TextIOWrapper(in_file, encoding="utf-8")
    
        game_count = 0
        loaded_count = 0
        game = ""
        
        with open(output_path, "w") as out_f:
            out_f.write(csv_headers)
            
            for line in text_stream:
                if not line.startswith(keep):
                    continue

                #we mark the beginning of the game
                elif line.startswith("[Site"):
                    game_count += 1 #how many games have we checked
                    if game_count % 100000 == 0:
                        print(f"Checked games: {game_count}, loaded {loaded_count}, time {(time.time() - start):.2f} seconds")

                    #If the game string is non empty (the game passed the criteria) we load the game
                    if game:
                        out_f.write(game + "\n")
                        game = "" #empty the game after loading

                        loaded_count += 1
    
                    #Initialize the game with the ID
                    game = extract_value(line).split("/")[-1]
        
                elif game:
                    if line.startswith("[WhiteElo"):
                        #check white elo
                        w_elo = extract_value(line)
                        if int(w_elo) >= min_elo:
                            #if the condition is satisfied, check black elo
                            next_line = next(text_stream)
                            b_elo = extract_value(next_line)
                            if int(b_elo) >= min_elo:
                                game += ";" + w_elo + ";" + b_elo
                            else:
                                game = ""
                        else:
                            game = ""
                    elif line.startswith("1."):
                        game += ";" + clean_moves(line)
                    else:
                        game += ";" + extract_value(line)

            if game: #load the last game if non empty
                loaded_count += 1
                out_f.write(game + "\n")
                
        print(f"Finished. Total checked: {game_count}, loaded: {loaded_count}, time: {(time.time() - start):.2f} seconds")  

In [72]:
extract_games(input_path, output_path, min_elo, keep, csv_headers)

Finished. Total checked: 1000, loaded: 845, time: 0.20 seconds


In [73]:
df = pd.read_csv(output_path, sep = ";")

In [74]:
df.head(5)

,ID,Result,WhiteElo,BlackElo,ECO,Opening,TimeControl,Termination,Moves
0,H9hHMIAr,0-1,1886,1896,B08,Pirc Defense: Classical Variation,180+2,Normal,1. e4 d6 2. d4 Nf6 3. Nc3 g6 4. Nf3 Bg7 5. Be3...
1,Ltfez6j6,0-1,2256,2713,E02,Catalan Opening: Open Defense,120+0,Time forfeit,1. d4 e6 2. c4 Nf6 3. g3 d5 4. Bg2 dxc4 5. Nc3...
2,ewaZsbtr,1/2-1/2,1924,1256,D02,Queen's Pawn Game: London System,600+0,Normal,1. Nf3 d5 2. d4 Nf6 3. Bf4 Bf5 4. g3 e6 5. Bg2...
3,JBS9P2bD,0-1,1386,2003,C54,"Italian Game: Classical Variation, Giuoco Pian...",600+0,Normal,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...
4,OptSyanm,1-0,1185,1321,C10,"French Defense: Rubinstein Variation, Ellis Ga...",600+0,Normal,1. e4 e6 2. d4 d5 3. Nc3 dxe4 4. Nxe4 e5 5. d5...


# Further Cleaning

## Transform the time control column

calculation: time + 40* (increment)
- < 29s = UltraBullet
- < 179s = Bullet
- < 479s = Blitz
- < 1499s = Rapid
- ≥ 1500s = Classical

In [75]:

def time_control(x):
    """ Assigns the time control cathegory to time control"""
    if "+" in x:
        time, inc = [int(i) for i in x.split("+")]
        sum_time = time + 40*inc
        if sum_time < 29:
            return "UltraBullet"
        elif sum_time < 179:
            return "Bullet"
        elif sum_time < 479:
            return "Blitz"
        elif sum_time < 1499:
            return "Rapid"
        else:
            return "Classical"
    else:
        return "NoTime"

In [76]:
df["TimeControl"] = df["TimeControl"].apply(time_control)
df.head(5)


,ID,Result,WhiteElo,BlackElo,ECO,Opening,TimeControl,Termination,Moves
0,H9hHMIAr,0-1,1886,1896,B08,Pirc Defense: Classical Variation,Blitz,Normal,1. e4 d6 2. d4 Nf6 3. Nc3 g6 4. Nf3 Bg7 5. Be3...
1,Ltfez6j6,0-1,2256,2713,E02,Catalan Opening: Open Defense,Bullet,Time forfeit,1. d4 e6 2. c4 Nf6 3. g3 d5 4. Bg2 dxc4 5. Nc3...
2,ewaZsbtr,1/2-1/2,1924,1256,D02,Queen's Pawn Game: London System,Rapid,Normal,1. Nf3 d5 2. d4 Nf6 3. Bf4 Bf5 4. g3 e6 5. Bg2...
3,JBS9P2bD,0-1,1386,2003,C54,"Italian Game: Classical Variation, Giuoco Pian...",Rapid,Normal,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...
4,OptSyanm,1-0,1185,1321,C10,"French Defense: Rubinstein Variation, Ellis Ga...",Rapid,Normal,1. e4 e6 2. d4 d5 3. Nc3 dxe4 4. Nxe4 e5 5. d5...


In [77]:
#check the result
freq_time = df["TimeControl"].value_counts()
freq_time

TimeControl
Blitz          413
Bullet         293
Rapid          124
Classical        8
UltraBullet      6
NoTime           1
Name: count, dtype: int64

## Remove Unterminated, Rules infraction and Abandoned games

In [78]:
freq_time = df["Termination"].value_counts()
freq_time

Termination
Normal          569
Time forfeit    263
Abandoned        13
Name: count, dtype: int64

In [79]:
df = df[~df["Termination"].isin(["Unterminated", "Rules infraction", "Abandoned"])].reset_index()

In [80]:
#check the result
freq_term = df["Termination"].value_counts()
freq_term

Termination
Normal          569
Time forfeit    263
Name: count, dtype: int64

## Clean Moves

In [81]:
df = df[~df["Moves"].isnull()].reset_index() #remove games with no moves

In [82]:
def remove_result(game_str):
    '''Removes the result from the game string'''
    tokens = game_str.strip().split(" ")
    if tokens[-1] in {'1-0', '0-1', '1/2-1/2', '*'}:
        tokens = tokens[:-1]
    return ' '.join(tokens)

In [83]:
df["Moves"] = df["Moves"].apply(remove_result)

In [84]:
df.to_csv(output_path, sep=";", index = False)

# Create fen moves datasets

In [85]:
def moves_to_fens(moves_str, max_moves = 20):
    """
    Given a move string, returns a fen representation of each position that occured in the first "max_moves" moves.
    """
    board = chess.Board()
    fens = []
    tokens = moves_str.split()

    move_count = 0
    for token in tokens:
        if token.endswith('.'):
            continue  # skip move numbers ("1.", "2.")
        try:
            board.push_san(token) #d4 –> d2d4
            fens.append(board.epd())
            move_count += 1
            if move_count >= max_moves * 2: #black and white move
                break
        except: #invalid move
            break 
    return fens


In [86]:
max_moves = 20
fen_lists = df["Moves"].apply(lambda m: moves_to_fens(m, max_moves = max_moves))

In [87]:
fen_moves= pd.concat([df["ID"], pd.DataFrame(fen_lists.tolist())], axis=1)

In [88]:
fen_moves.head(5)

,ID,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,H9hHMIAr,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,rnbqkbnr/ppp1pppp/3p4/8/4P3/8/PPPP1PPP/RNBQKBN...,rnbqkbnr/ppp1pppp/3p4/8/3PP3/8/PPP2PPP/RNBQKBN...,rnbqkb1r/ppp1pppp/3p1n2/8/3PP3/8/PPP2PPP/RNBQK...,rnbqkb1r/ppp1pppp/3p1n2/8/3PP3/2N5/PPP2PPP/R1B...,rnbqkb1r/ppp1pp1p/3p1np1/8/3PP3/2N5/PPP2PPP/R1...,rnbqkb1r/ppp1pp1p/3p1np1/8/3PP3/2N2N2/PPP2PPP/...,rnbqk2r/ppp1ppbp/3p1np1/8/3PP3/2N2N2/PPP2PPP/R...,rnbqk2r/ppp1ppbp/3p1np1/8/3PP3/2N1BN2/PPP2PPP/...,...,r2q1rk1/pppb1nbp/3p1pn1/3Pp1p1/4P1PP/2N3B1/PPP...,r2q1rk1/pppb1nb1/3p1pnp/3Pp1p1/4P1PP/2N3B1/PPP...,r2q1rk1/pppb1nb1/3p1pnp/3Pp1P1/4P1P1/2N3B1/PPP...,r2q1rk1/pppb1nb1/3p2np/3Pp1p1/4P1P1/2N3B1/PPPQ...,r2q1rk1/pppb1nb1/3p2np/3Pp1p1/4P1P1/2N3B1/PPPQ...,r2q1rk1/pppb1nb1/3p3p/3Pp1p1/4PnP1/2N3B1/PPPQB...,r2q1rk1/pppb1nb1/3p3p/3Pp1p1/4PBP1/2N5/PPPQBP1...,r2q1rk1/pppb1nb1/3p3p/3P2p1/4PpP1/2N5/PPPQBP1N...,r2q1rk1/pppb1nb1/3p3p/3P2p1/4PpP1/2N2P2/PPPQB2...,r2q1rk1/pppb2b1/3p3p/3Pn1p1/4PpP1/2N2P2/PPPQB2...
1,Ltfez6j6,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBN...,rnbqkbnr/pppp1ppp/4p3/8/2PP4/8/PP2PPPP/RNBQKBN...,rnbqkb1r/pppp1ppp/4pn2/8/2PP4/8/PP2PPPP/RNBQKB...,rnbqkb1r/pppp1ppp/4pn2/8/2PP4/6P1/PP2PP1P/RNBQ...,rnbqkb1r/ppp2ppp/4pn2/3p4/2PP4/6P1/PP2PP1P/RNB...,rnbqkb1r/ppp2ppp/4pn2/3p4/2PP4/6P1/PP2PPBP/RNB...,rnbqkb1r/ppp2ppp/4pn2/8/2pP4/6P1/PP2PPBP/RNBQK...,rnbqkb1r/ppp2ppp/4pn2/8/2pP4/2N3P1/PP2PPBP/R1B...,...,1rbr2k1/2p1qppp/p1B1p3/1pn1P3/2p1NP2/4PQP1/PP5...,1rbr2k1/2p1qppp/p1B1p3/1p2P3/2p1NP2/3nPQP1/PP5...,1rbr2k1/2p1qppp/p1B1p3/1p2P3/2p1NP2/3nPQP1/PP5...,1rbr2k1/2p1qppp/p1B1p3/1p2P3/2p1NP2/4PQP1/Pn5P...,1rbR2k1/2p1qppp/p1B1p3/1p2P3/2p1NP2/4PQP1/Pn5P...,1rbq2k1/2p2ppp/p1B1p3/1p2P3/2p1NP2/4PQP1/Pn5P/...,1rbq2k1/2p2ppp/p1B1p3/1p2P1N1/2p2P2/4PQP1/Pn5P...,1rbq2k1/2p2pp1/p1B1p2p/1p2P1N1/2p2P2/4PQP1/Pn5...,1rbq2k1/2p2pp1/p1B1p2p/1p2P3/2p1NP2/4PQP1/Pn5P...,1rbq2k1/2p2pp1/p1B1p2p/4P3/1pp1NP2/4PQP1/Pn5P/...
2,ewaZsbtr,rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R ...,rnbqkbnr/ppp1pppp/8/3p4/8/5N2/PPPPPPPP/RNBQKB1...,rnbqkbnr/ppp1pppp/8/3p4/3P4/5N2/PPP1PPPP/RNBQK...,rnbqkb1r/ppp1pppp/5n2/3p4/3P4/5N2/PPP1PPPP/RNB...,rnbqkb1r/ppp1pppp/5n2/3p4/3P1B2/5N2/PPP1PPPP/R...,rn1qkb1r/ppp1pppp/5n2/3p1b2/3P1B2/5N2/PPP1PPPP...,rn1qkb1r/ppp1pppp/5n2/3p1b2/3P1B2/5NP1/PPP1PP1...,rn1qkb1r/ppp2ppp/4pn2/3p1b2/3P1B2/5NP1/PPP1PP1...,rn1qkb1r/ppp2ppp/4pn2/3p1b2/3P1B2/5NP1/PPP1PPB...,...,r4rk1/pp1n1ppp/2p1pq2/8/3P4/1Q3NP1/P4PBP/R3R1K...,r4rk1/pp3ppp/1np1pq2/8/3P4/1Q3NP1/P4PBP/R3R1K1...,r4rk1/pp3ppp/1np1pq2/8/P2P4/1Q3NP1/5PBP/R3R1K1...,1r3rk1/pp3ppp/1np1pq2/8/P2P4/1Q3NP1/5PBP/R3R1K...,1r3rk1/pp3ppp/1np1pq2/P7/3P4/1Q3NP1/5PBP/R3R1K...,1r3rk1/pp3ppp/2p1pq2/P2n4/3P4/1Q3NP1/5PBP/R3R1...,1r3rk1/pp3ppp/2p1pq2/P2nN3/3P4/1Q4P1/5PBP/R3R1...,1r1r2k1/pp3ppp/2p1pq2/P2nN3/3P4/1Q4P1/5PBP/R3R...,1r1r2k1/pp3ppp/2p1pq2/P2BN3/3P4/1Q4P1/5P1P/R3R...,1r4k1/pp3ppp/2p1pq2/P2rN3/3P4/1Q4P1/5P1P/R3R1K...
3,JBS9P2bD,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/R...,r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP...,r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/3P1N2/PPP2PP...,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/3P1N2/PPP2P...,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2PP1N2/PP3P...,...,r2qr1k1/1pp2pp1/2np3p/p6b/P2PPn2/1B2RN1P/1P3PP...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PPn2/1B2RN1P/1P3P...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PPnP1/1B2RN1P/1P3...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1P1/1B2RN1n/1P3...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1P1/1B2RN1n/1P3...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1P1/1B2RN2/1P3n...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1P1/1B2RN2/1P3n...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1n1/1B2RN2/1P5K...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1n1/1B2RN1K/1P6...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP3/1B2nN1K/1P6/R...
4,OptSyanm,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,rnbqkbnr/pppp1ppp/4p3/8

## Separate the positons to: black to move and white to move

In [89]:
#white to move columns [1, 3, 5, 7, ....]
#black to move columns  [0, 2, 4, ....]
white_to_move = ["ID"]
black_to_move = ["ID"]
for i in range(0, 2*max_moves):
    if i % 2 == 0:
        black_to_move.append(i)
    else:
        white_to_move.append(i)



In [90]:
white_to_move_fens = fen_moves[white_to_move]
black_to_move_fens = fen_moves[black_to_move]

In [91]:
white_to_move_fens.head(5)

,ID,1,3,5,7,9,11,13,15,17,...,21,23,25,27,29,31,33,35,37,39
0,H9hHMIAr,rnbqkbnr/ppp1pppp/3p4/8/4P3/8/PPPP1PPP/RNBQKBN...,rnbqkb1r/ppp1pppp/3p1n2/8/3PP3/8/PPP2PPP/RNBQK...,rnbqkb1r/ppp1pp1p/3p1np1/8/3PP3/2N5/PPP2PPP/R1...,rnbqk2r/ppp1ppbp/3p1np1/8/3PP3/2N2N2/PPP2PPP/R...,rnbq1rk1/ppp1ppbp/3p1np1/8/3PP3/2N1BN2/PPP2PPP...,rnbq1rk1/ppp1ppbp/3p2p1/8/3PP1n1/2N1BN2/PPPQ1P...,rnbq1rk1/ppp2pbp/3p2p1/4p3/3PPBn1/2N2N2/PPPQ1P...,rnbq1rk1/ppp3bp/3p1pp1/4p1B1/3PP1n1/2N2N2/PPPQ...,r1bq1rk1/ppp3bp/2np1pp1/4p3/3PP1nB/2N2N2/PPPQ1...,...,r1bq1rk1/ppp1n1bp/3p1ppn/3Pp3/4P2B/2N2N1P/PPPQ...,r1bq1rk1/ppp1n1bp/3p1p1n/3Pp1p1/4P1PB/2N2N1P/P...,r1bq1rk1/ppp3bp/3p1pnn/3Pp1p1/4P1P1/2N2NBP/PPP...,r2q1rk1/pppb2bp/3p1pnn/3Pp1p1/4P1P1/2N2NBP/PPP...,r2q1rk1/pppb1nbp/3p1pn1/3Pp1p1/4P1P1/2N3BP/PPP...,r2q1rk1/pppb1nb1/3p1pnp/3Pp1p1/4P1PP/2N3B1/PPP...,r2q1rk1/pppb1nb1/3p2np/3Pp1p1/4P1P1/2N3B1/PPPQ...,r2q1rk1/pppb1nb1/3p3p/3Pp1p1/4PnP1/2N3B1/PPPQB...,r2q1rk1/pppb1nb1/3p3p/3P2p1/4PpP1/2N5/PPPQBP1N...,r2q1rk1/pppb2b1/3p3p/3Pn1p1/4PpP1/2N2P2/PPPQB2...
1,Ltfez6j6,rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBN...,rnbqkb1r/pppp1ppp/4pn2/8/2PP4/8/PP2PPPP/RNBQKB...,rnbqkb1r/ppp2ppp/4pn2/3p4/2PP4/6P1/PP2PP1P/RNB...,rnbqkb1r/ppp2ppp/4pn2/8/2pP4/6P1/PP2PPBP/RNBQK...,r1bqkb1r/ppp2ppp/2n1pn2/8/2pP4/2N3P1/PP2PPBP/R...,1rbqkb1r/ppp2ppp/2n1pn2/8/2pP4/2N2NP1/PP2PPBP/...,1rbqkb1r/p1p2ppp/2n1pn2/1p6/2pP4/2N2NP1/PP2PPB...,1rbqk2r/p1p1bppp/2n1pn2/1p4B1/2pP4/2N2NP1/PP2P...,1rbqk2r/p1p1bppp/4pn2/1p2n1B1/2pP4/2N3P1/PP2PP...,...,1rb1k2r/p1pnqppp/4p3/1p2P3/2p5/2N3P1/PP2PPBP/R...,1rb1k2r/2pnqppp/p1B1p3/1p2P3/2p5/2N3P1/PP2PP1P...,1rb2rk1/2pnqppp/p1B1p3/1p2P3/2p1N3/6P1/PP2PP1P...,1rb2rk1/2p1qppp/p1B1p3/1pn1P3/2p1NP2/6P1/PP2P2...,1rbr2k1/2p1qppp/p1B1p3/1pn1P3/2p1NP2/4P1P1/PP5...,1rbr2k1/2p1qppp/p1B1p3/1p2P3/2p1NP2/3nPQP1/PP5...,1rbr2k1/2p1qppp/p1B1p3/1p2P3/2p1NP2/4PQP1/Pn5P...,1rbq2k1/2p2ppp/p1B1p3/1p2P3/2p1NP2/4PQP1/Pn5P/...,1rbq2k1/2p2pp1/p1B1p2p/1p2P1N1/2p2P2/4PQP1/Pn5...,1rbq2k1/2p2pp1/p1B1p2p/4P3/1pp1NP2/4PQP1/Pn5P/...
2,ewaZsbtr,rnbqkbnr/ppp1pppp/8/3p4/8/5N2/PPPPPPPP/RNBQKB1...,rnbqkb1r/ppp1pppp/5n2/3p4/3P4/5N2/PPP1PPPP/RNB...,rn1qkb1r/ppp1pppp/5n2/3p1b2/3P1B2/5N2/PPP1PPPP...,rn1qkb1r/ppp2ppp/4pn2/3p1b2/3P1B2/5NP1/PPP1PP1...,rn1qk2r/ppp2ppp/3bpn2/3p1b2/3P1B2/5NP1/PPP1PPB...,rn2k2r/ppp2ppp/3qpn2/3p1b2/3P4/5NP1/PPP1PPBP/R...,rn3rk1/ppp2ppp/3qpn2/3p1b2/3P4/5NP1/PPP1PPBP/R...,rn3rk1/ppp2ppp/3qpn2/5b2/2pP4/5NP1/PP2PPBP/RN1...,rn3rk1/pp3ppp/2pqpn2/5b2/2pP4/5NP1/PP1NPPBP/R2...,...,r4rk1/pp1n1ppp/2pqpn2/8/2pPP1b1/5NP1/PP1N1PBP/...,r4rk1/pp1n1ppp/2pqpn2/4P3/2pP4/5bP1/PP1N1PBP/R...,r4rk1/pp1nqppp/2p1pn2/4P3/2pP4/5NP1/PP3PBP/R2Q...,r4rk1/pp1n1ppp/2p1pq2/8/2pP4/5NP1/PP3PBP/R2QR1...,r4rk1/pp1n1ppp/2p1pq2/8/3P4/1p3NP1/P4PBP/R2QR1...,r4rk1/pp3ppp/1np1pq2/8/3P4/1Q3NP1/P4PBP/R3R1K1...,1r3rk1/pp3ppp/1np1pq2/8/P2P4/1Q3NP1/5PBP/R3R1K...,1r3rk1/pp3ppp/2p1pq2/P2n4/3P4/1Q3NP1/5PBP/R3R1...,1r1r2k1/pp3ppp/2p1pq2/P2nN3/3P4/1Q4P1/5PBP/R3R...,1r4k1/pp3ppp/2p1pq2/P2rN3/3P4/1Q4P1/5P1P/R3R1K...
3,JBS9P2bD,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP...,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/3P1N2/PPP2P...,r1bqk2r/ppp2ppp/2np1n2/2b1p3/2B1P3/2PP1N2/PP3P...,r1bq1rk1/ppp2ppp/2np1n2/2b1p3/4P3/1BPP1N2/PP3P...,r1bq1rk1/1pp2ppp/2np1n2/p1b1p3/4P3/1BPP1N2/PP3...,r1bq1rk1/1pp2pp1/2np1n1p/p1b1p3/P3P3/1BPP1N2/1...,r1bqr1k1/1pp2pp1/2np1n1p/p1b1p3/P3P3/1BPP1N2/1...,...,r1bqr1k1/1pp2pp1/2np3p/p3p2n/P3P3/1BPPRN2/1P3P...,r1bqr1k1/1pp2pp1/2np3p/p3p3/P2PPn2/1BP1RN2/1P3...,r2qr1k1/1pp2pp1/2np3p/p3p3/P2PPnb1/NBP1RN2/1P3...,r2qr1k1/1pp2pp1/2np3p/p7/P2pPnb1/NBP1RN1P/1P3P...,r2qr1k1/1pp2pp1/2np3p/p6b/P2PPn2/NB2RN1P/1P3PP...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PPn2/1B2RN1P/1P3P...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1P1/1B2RN1n/1P3...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1P1/1B2RN2/1P3n...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP1n1/1B2RN2/1P5K...,r3r1k1/1pp2pp1/2np1q1p/p6b/P2PP3/1B2nN1K/1P6/R...
4,OptSyanm,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,rnbqkbnr/ppp2ppp/4p

rename the columns to the number of the next move
- for white [2,3,4,21]
- for black [1,2...20] <br>

e.g. in the black_to_move_fens in the column named 2 we have a positon in which black is to move his 2nd move

In [92]:
white_to_move_fens_cols = ["ID"] + [i for i in range(2, max_moves + 2)]
black_to_move_fens_cols = ["ID"] + [i for i in range(1, max_moves + 1)]


In [93]:
white_to_move_fens.columns = white_to_move_fens_cols
black_to_move_fens.columns = black_to_move_fens_cols

In [94]:
white_to_move_fens.to_csv("white_to_move_fen_sample.csv", sep=";", index = False)
black_to_move_fens.to_csv("black_to_move_fens_sample.csv", sep=";", index = False)